# Set up

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
import requests
from config import db_login_info

## Get NYT county COVID data

In [2]:
# -----------------------
# This is a NYTimes report on national COVID-19 cases and deaths, sorted by county
# From: https://github.com/nytimes/covid-19-data (us-counties.csv)
# -----------------------

# Pull in the live data
counties_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

# Save to a dataframe
covid_all_df = pd.read_csv(counties_path)

#### Build all values dataframe

In [3]:
# Drop rows with null values (we want complete data only)
covid_all_df = covid_all_df.dropna()

# Convert fips to int
covid_all_df = covid_all_df.astype({'fips': 'int'})

In [4]:
# Display the dataframe
covid_all_df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
615474,2020-10-09,Sweetwater,Wyoming,56037,393,2
615475,2020-10-09,Teton,Wyoming,56039,656,1
615476,2020-10-09,Uinta,Wyoming,56041,378,2
615477,2020-10-09,Washakie,Wyoming,56043,122,6


#### Build latest values dataframe

In [5]:
# Get the most up-to-date data
latest_date = covid_all_df['date'].iloc[-1]

# Make a dataframe holding that most recent data
covid_latest_df = covid_all_df.loc[covid_all_df['date']==latest_date]

In [6]:
# Display the dataframe
covid_latest_df

,date,county,state,fips,cases,deaths
612238,2020-10-09,Autauga,Alabama,1001,1882,28
612239,2020-10-09,Baldwin,Alabama,1003,6172,64
612240,2020-10-09,Barbour,Alabama,1005,939,9
612241,2020-10-09,Bibb,Alabama,1007,719,12
612242,2020-10-09,Blount,Alabama,1009,1689,16
...,...,...,...,...,...,...
615474,2020-10-09,Sweetwater,Wyoming,56037,393,2
615475,2020-10-09,Teton,Wyoming,56039,656,1
615476,2020-10-09,Uinta,Wyoming,56041,378,2
615477,2020-10-09,Washakie,Wyoming,56043,122,6


## Get mask use data

In [7]:
# Import the mask CSV data
mask_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv"

# Keep the leading 0 for fips
masks_df = pd.read_csv(mask_path, converters={'COUNTYFP': lambda x: str(x)})

In [8]:
# Clean up column names
masks_df.columns = ['fips','never','rarely','sometimes','frequently','always']

In [9]:
# Display the dataframe
masks_df

,fips,never,rarely,sometimes,frequently,always
0,01001,0.053,0.074,0.134,0.295,0.444
1,01003,0.083,0.059,0.098,0.323,0.436
2,01005,0.067,0.121,0.120,0.201,0.491
3,01007,0.020,0.034,0.096,0.278,0.572
4,01009,0.053,0.114,0.180,0.194,0.459
...,...,...,...,...,...,...
3137,56037,0.061,0.295,0.230,0.146,0.268
3138,56039,0.095,0.157,0.160,0.247,0.340
3139,56041,0.098,0.278,0.154,0.207,0.264
3140,56043,0.204,0.155,0.069,0.285,0.287


## Get CDC data
#### Source: https://data.cdc.gov/NCHS/Weekly-Counts-of-Deaths-by-State-and-Select-Causes/muzy-jte6

In [10]:
# Convert to a dataframe
cdc_df = pd.read_csv("../data/Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2019-2020.csv")

In [11]:
# Drop columns we don't need
cdc_df.drop(list(cdc_df)[19:34], axis=1,inplace=True)

# Rename columns
cdc_df.columns = ['state',
                  'year',
                  'week',
                  'week_ending_date',
                  'all_causes',
                  'natural_causes',
                  'septicemia',
                  'malignant_neoplasms',
                  'diabetes',
                  'alzheimers',
                  'influenza_and_pneumonia',
                  'chronic_lower_respiratory',
                  'other_diseases_of_respiratory',
                  'nephritis_nephrotic_syndrome',
                  'symptoms_signs_and_abnormal',
                  'diseases_of_heart',
                  'cerebrovascular_diseases',
                  'covid_19_multiple_causes',
                  'covid_19_underlying_cause']     


# Change the null values (no data) to 0
cdc_df = cdc_df.fillna(0)

In [12]:
# Display the dataframe
cdc_df

,state,year,week,week_ending_date,all_causes,natural_causes,septicemia,malignant_neoplasms,diabetes,alzheimers,influenza_and_pneumonia,chronic_lower_respiratory,other_diseases_of_respiratory,nephritis_nephrotic_syndrome,symptoms_signs_and_abnormal,diseases_of_heart,cerebrovascular_diseases,covid_19_multiple_causes,covid_19_underlying_cause
0,Alabama,2019,1,2019-01-05,1077.0,993.0,30.0,198.0,22.0,60.0,21.0,63.0,14.0,21.0,27.0,261.0,53.0,0.0,0.0
1,Alabama,2019,2,2019-01-12,1090.0,994.0,25.0,187.0,24.0,49.0,18.0,85.0,21.0,13.0,11.0,275.0,65.0,0.0,0.0
2,Alabama,2019,3,2019-01-19,1114.0,1042.0,22.0,238.0,18.0,48.0,31.0,80.0,30.0,25.0,15.0,283.0,53.0,0.0,0.0
3,Alabama,2019,4,2019-01-26,1063.0,994.0,21.0,165.0,22.0,50.0,22.0,113.0,14.0,25.0,23.0,279.0,56.0,0.0,0.0
4,Alabama,2019,5,2019-02-02,1095.0,1026.0,18.0,199.0,19.0,52.0,19.0,80.0,20.0,24.0,21.0,290.0,50.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4901,United States,2020,35,2020-08-29,55315.0,52044.0,658.0,10781.0,1620.0,2260.0,617.0,2327.0,678.0,890.0,3028.0,10795.0,2733.0,5106.0,4587.0
4902,United States,2020,36,2020-09-05,52108.0,49073.0,610.0,10179.0,1504.0,2266.0,626.0,2213.0,698.0,802.0,2971.0,10429.0,2515.0,4162.0,3668.0
4903,United States,2020,37,2020-09-12,48078.0,45611.0,549.0,9723.0,1403.0,2025.0,567.0,2093.0,601.0,754.0,3041.0,9533.0,2526.0,3313.0,2923.0
4904,United States,2020,38,2020-09-19,38755.0,37168.0,459.0,8323.0,1105.0,1780.0,465.0,1742.0,533.0,600.0,2372.0,8047.0,1992.0,1902.0,1680.0


## Get Covid Tracking Project data
Source: https://covidtracking.com

#### Build latest values dataframe

In [13]:
covidtracking_current_url = 'https://api.covidtracking.com/v1/states/current.json'
covidtracking_current_response = requests.get(covidtracking_current_url)
covidtracking_current_json = covidtracking_current_response.json()

# Convert to a dataframe
covidtracking_current_df = pd.DataFrame.from_dict(covidtracking_current_json)

In [14]:
# Change the null values (no data) to 0
covidtracking_current_df = covidtracking_current_df.fillna(0)

In [15]:
# Remove columns we don't need
covidtracking_current_df.drop(list(covidtracking_current_df)[48:55], axis=1,inplace=True)
covidtracking_current_df.drop(list(covidtracking_current_df)[39], axis=1,inplace=True)
covidtracking_current_df.drop(list(covidtracking_current_df)[20], axis=1,inplace=True)
covidtracking_current_df.drop(list(covidtracking_current_df)[15:18], axis=1,inplace=True)

In [16]:
covidtracking_current_df.columns = ['date', 'state', 'positive', 'probable_cases', 'negative', 'pending',
       'total_test_results', 'hospitalized_currently', 'hospitalized_cumulative',
       'icu_currently', 'icu_cumulative', 'ventilator_currently',
       'ventilator_cumulative', 'recovered', 'data_quality_grade', 'deaths',
       'hospitalized', 'total_tests_viral', 'positive_tests_viral',
       'negative_tests_viral', 'positive_cases_viral', 'deaths_confirmed',
       'deaths_probable', 'total_test_encounters_viral', 'total_tests_people_viral',
       'total_tests_antibody', 'positive_tests_antibody', 'negative_tests_antibody',
       'total_tests_people_antibody', 'positive_tests_people_antibody',
       'negative_tests_people_antibody', 'total_tests_people_antigen',
       'positive_tests_people_antigen', 'total_tests_antigen',
       'positive_tests_antigen', 'positive_increase', 'negative_increase', 'total',
       'total_test_results_source', 'total_test_results_increase', 'pos_neg',
       'death_increase', 'hospitalized_increase']

In [17]:
# Display the dataframe
covidtracking_current_df

,date,state,positive,probable_cases,negative,pending,total_test_results,hospitalized_currently,hospitalized_cumulative,icu_currently,...,total_tests_antigen,positive_tests_antigen,positive_increase,negative_increase,total,total_test_results_source,total_test_results_increase,pos_neg,death_increase,hospitalized_increase
0,20201009,AK,10176,0.0,482894,0.0,493070,51.0,0.0,0.0,...,0.0,0.0,180,1719,493070,posNeg,1899,493070,0,0
1,20201009,AL,163465,19565.0,1052552,0.0,1196452,816.0,17989.0,0.0,...,0.0,0.0,1490,10506,1216017,posNeg,11742,1216017,16,0
2,20201009,AR,90145,4165.0,1025546,0.0,1111526,546.0,5805.0,237.0,...,21856.0,3300.0,0,0,1115691,posNeg,0,1115691,0,0
3,20201009,AS,0,0.0,1616,0.0,1616,0.0,0.0,0.0,...,0.0,0.0,0,0,1616,posNeg,0,1616,0,0
4,20201009,AZ,224084,4837.0,1324198,0.0,1543445,706.0,20199.0,155.0,...,0.0,0.0,683,11699,1548282,posNeg,12350,1548282,3,28
5,20201009,CA,838606,0.0,14897891,0.0,15736497,3186.0,0.0,742.0,...,0.0,0.0,3806,109068,15736497,posNeg,112874,15736497,67,0
6,20201009,CO,75785,5226.0,914437,0.0,1496198,356.0,7834.0,0.0,...,0.0,0.0,863,9322,990222,totalTestEncountersViral,18897,990222,7,63
7,20201009,CT,60038,2440.0,1736501,0.0,1796539,134.0,11845.0,0.0,...,0.0,0.0,290,16991,1796539,posNeg,17281,1796539,3,0
8,20201009,DC,15843,0.0,406382,0.0,422225,99.0,0.0,27.0,...,0.0,0.0,78,5331,422225,posNeg,5409,422225,0,0
9,20201009,DE,21827,1079.0,282540,0.0,304367,104.0,0.0,21.0,...,0.0,0.0,145,2720,304367,posNeg,2865,304367,0,0


#### Build all values dataframe

In [18]:
covidtracking_all_url = 'https://api.covidtracking.com/v1/us/daily.json'
covidtracking_all_response = requests.get(covidtracking_all_url)
covidtracking_all_json = covidtracking_all_response.json()

# Convert to a dataframe
covidtracking_all_df = pd.DataFrame.from_dict(covidtracking_all_json)

In [19]:
# Change the null values (no data) to 0
covidtracking_all_df = covidtracking_all_df.fillna(0)

In [20]:
# Remove columns we don't need
covidtracking_all_df.drop(list(covidtracking_all_df)[24], axis=1,inplace=True)
covidtracking_all_df.drop(list(covidtracking_all_df)[16], axis=1,inplace=True)
covidtracking_all_df.drop(list(covidtracking_all_df)[12], axis=1,inplace=True)

In [21]:
covidtracking_all_df.columns = ['date', 'states', 'positive', 'negative', 'pending',
       'hospitalized_currently', 'hospitalized_cumulative', 'icu_currently',
       'icu_cumulative', 'ventilator_currently', 'ventilator_cumulative',
       'recovered', 'deaths', 'hospitalized', 'total_test_results', 'total',
       'pos_neg', 'death_increase', 'hospitalized_increase', 'negative_increase',
       'positive_increase', 'total_test_results_increase']

In [22]:
# Display the dataframe
covidtracking_all_df

,date,states,positive,negative,pending,hospitalized_currently,hospitalized_cumulative,icu_currently,icu_cumulative,ventilator_currently,...,deaths,hospitalized,total_test_results,total,pos_neg,death_increase,hospitalized_increase,negative_increase,positive_increase,total_test_results_increase
0,20201009,56,7623648,100497078,13078.0,34839.0,420587.0,6713.0,21389.0,1655.0,...,205470.0,420587.0,113347658,0,0,904,1824,956046,57542,1098217
1,20201008,56,7566106,99541032,13087.0,34322.0,418763.0,6621.0,21217.0,1638.0,...,204566.0,418763.0,112249441,0,0,975,2052,875485,54870,1034249
2,20201007,56,7511236,98665547,12269.0,32105.0,416711.0,6507.0,21112.0,1646.0,...,203591.0,416711.0,111215192,0,0,916,2250,732692,50602,862764
3,20201006,56,7460634,97932855,8680.0,31346.0,414461.0,6438.0,20973.0,1609.0,...,202675.0,414461.0,110352428,0,0,634,-624,722475,38661,824253
4,20201005,56,7421973,97210380,11544.0,30098.0,415085.0,6209.0,20812.0,1515.0,...,202041.0,415085.0,109528175,0,0,326,1519,846256,38133,937079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,20200126,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,0
258,20200125,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,0
259,20200124,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,0
260,20200123,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,1


## Get the county information table

In [23]:
# Scrape the county info table from Wikipedia
county_url = 'https://en.wikipedia.org/wiki/User:Michael_J/County_table'
county_table = pd.read_html(county_url)

In [24]:
# Grab the first table on the page and convert to dataframe
county_table_df = county_table[0]

# Drop columns we don't need
county_table_df = county_table_df.drop(columns=['Land Areakm²','Land Areami²','Water Areakm²','Water Areami²','Total Areakm²','Total Areami²','Sort [1]','Population(2010)'])

# Rename the columns
county_table_df.columns = ['state','fips','county','county_seat','lat','lon']

# Remove the + sign from latitude column
county_table_df['lat'] = county_table_df['lat'].str[1:]
county_table_df['lon'] = county_table_df['lon'].str[1:]

# Remove degree symbol from lat and lon
county_table_df['lat'] = county_table_df['lat'].str[:-1]
county_table_df['lon'] = county_table_df['lon'].str[:-1]

# Set as a float for lat and lon
county_table_df['lat'] = county_table_df['lat'].astype(float)
county_table_df['lon'] = county_table_df['lon'].astype(float)

# Convert the lon to an actual negative value (for all, since North America)
county_table_df['lon'] = -county_table_df['lon']

# Change the null values (no data) to 0
county_table_df = county_table_df.fillna(0)

In [25]:
# Display the dataframe
county_table_df

,state,fips,county,county_seat,lat,lon
0,AL,1001,Autauga,Prattville,32.536382,-86.644490
1,AL,1003,Baldwin,Bay Minette,30.659218,-87.746067
2,AL,1005,Barbour,Clayton,31.870670,-85.405456
3,AL,1007,Bibb,Centreville,33.015893,-87.127148
4,AL,1009,Blount,Oneonta,33.977448,-86.567246
...,...,...,...,...,...,...
3138,WY,56037,Sweetwater,Green River,41.660339,-108.875676
3139,WY,56039,Teton,Jackson,44.049321,-110.588102
3140,WY,56041,Uinta,Evanston,41.284726,-110.558947
3141,WY,56043,Washakie,Worland,43.878831,-107.669052


## Connect and load to postgressql
### Available dataframes:
<b>NYT:</b> covid_all_df, covid_latest_df <br>
<b>Masks:</b> masks_df <br>
<b>CDC:</b> cdc_df, cdc_2019_df, cdc_2020_df <br>
<b>Covid tracking:</b> covidtracking_current_df, covidtracking_all_df<br>
<b>County info:</b> county_table_df

In [26]:
# Connect to database
#
# Note: db_login_info comes from config.py and should have a scheme of:
# "postgresql://<USERNAME>:<PASSWORD>@<HOST>:<PORT>/<DATABASE_NAME>"

engine = create_engine(f'{db_login_info}')

In [27]:
# Check for Tables
table_names = engine.table_names()
table_names

['county',
 'covid',
 'covidtracking_current',
 'covidtracking_all',
 'masks',
 'cdc']

In [28]:
engine.dispose()

### Load data into database
These are commented out to prevent damaging the existing tables host on Heroku

In [29]:
# Load covid_all_df dataframe into database
#covid_all_df.to_sql(name='covid', con=engine, if_exists='append', index=False)

In [30]:
# Load mask_df dataframe into database
#masks_df.to_sql(name='masks', con=engine, if_exists='append', index=False)

In [31]:
# Load cdc_df dataframe into database
#cdc_df.to_sql(name='cdc', con=engine, if_exists='append', index=False)

In [32]:
# Load covidtracking_current_df dataframe into database
#covidtracking_current_df.to_sql(name='covidtracking_current', con=engine, if_exists='append', index=False)

In [33]:
# Load covidtracking_all_df dataframe into database
#covidtracking_all_df.to_sql(name='covidtracking_all', con=engine, if_exists='append', index=False)

In [34]:
# Load county_table_df dataframe into database
#county_table_df.to_sql(name='county', con=engine, if_exists='append', index=False)

### Confirm data load

In [35]:
# Confirm covid data has been added
pd.read_sql_query('select * from covid', con=engine)

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
596706,2020-10-05,Sweetwater,Wyoming,56037,363,2
596707,2020-10-05,Teton,Wyoming,56039,625,1
596708,2020-10-05,Uinta,Wyoming,56041,365,2
596709,2020-10-05,Washakie,Wyoming,56043,120,6


In [36]:
# Confirm mask use data has been added
pd.read_sql_query('select * from masks', con=engine)

,fips,never,rarely,sometimes,frequently,always
0,01001,0.053,0.074,0.134,0.295,0.444
1,01003,0.083,0.059,0.098,0.323,0.436
2,01005,0.067,0.121,0.120,0.201,0.491
3,01007,0.020,0.034,0.096,0.278,0.572
4,01009,0.053,0.114,0.180,0.194,0.459
...,...,...,...,...,...,...
3137,56037,0.061,0.295,0.230,0.146,0.268
3138,56039,0.095,0.157,0.160,0.247,0.340
3139,56041,0.098,0.278,0.154,0.207,0.264
3140,56043,0.204,0.155,0.069,0.285,0.287


In [37]:
# Confirm cdc data has been added
pd.read_sql_query('select * from cdc', con=engine)

,state,year,week,week_ending_date,all_causes,natural_causes,septicemia,malignant_neoplasms,diabetes,alzheimers,influenza_and_pneumonia,chronic_lower_respiratory,other_diseases_of_respiratory,nephritis_nephrotic_syndrome,symptoms_signs_and_abnormal,diseases_of_heart,cerebrovascular_diseases,covid_19_multiple_causes,covid_19_underlying_cause
0,Alabama,2019,1,2019-01-05,1077,993,30,198,22,60,21,63,14,21,27,261,53,0,0
1,Alabama,2019,2,2019-01-12,1090,994,25,187,24,49,18,85,21,13,11,275,65,0,0
2,Alabama,2019,3,2019-01-19,1114,1042,22,238,18,48,31,80,30,25,15,283,53,0,0
3,Alabama,2019,4,2019-01-26,1063,994,21,165,22,50,22,113,14,25,23,279,56,0,0
4,Alabama,2019,5,2019-02-02,1095,1026,18,199,19,52,19,80,20,24,21,290,50,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4901,United States,2020,35,2020-08-29,55315,52044,658,10781,1620,2260,617,2327,678,890,3028,10795,2733,5106,4587
4902,United States,2020,36,2020-09-05,52108,49073,610,10179,1504,2266,626,2213,698,802,2971,10429,2515,4162,3668
4903,United States,2020,37,2020-09-12,48078,45611,549,9723,1403,2025,567,2093,601,754,3041,9533,2526,3313,2923
4904,United States,2020,38,2020-09-19,38755,37168,459,8323,1105,1780,465,1742,533,600,2372,8047,1992,1902,1680


In [38]:
# Confirm county data has been added
pd.read_sql_query('select * from county', con=engine)

,state,fips,county,county_seat,lat,lon
0,AL,1001,Autauga,Prattville,32.536382,-86.644490
1,AL,1003,Baldwin,Bay Minette,30.659218,-87.746067
2,AL,1005,Barbour,Clayton,31.870670,-85.405456
3,AL,1007,Bibb,Centreville,33.015893,-87.127148
4,AL,1009,Blount,Oneonta,33.977448,-86.567246
...,...,...,...,...,...,...
3138,WY,56037,Sweetwater,Green River,41.660339,-108.875676
3139,WY,56039,Teton,Jackson,44.049321,-110.588102
3140,WY,56041,Uinta,Evanston,41.284726,-110.558947
3141,WY,56043,Washakie,Worland,43.878831,-107.669052


In [39]:
# Confirm covidtracking_all data has been added
pd.read_sql_query('select * from covidtracking_all', con=engine)

,date,states,positive,negative,pending,hospitalized_currently,hospitalized_cumulative,icu_currently,icu_cumulative,ventilator_currently,...,deaths,hospitalized,total_test_results,total,pos_neg,death_increase,hospitalized_increase,negative_increase,positive_increase,total_test_results_increase
0,20201006,56,7460634,97932855,8680,31346,414461,6438,20973,1609,...,202675,414461,110226302,0,0,634,-624,722475,38661,823419
1,20201005,56,7421973,97210380,11544,30098,415085,6209,20812,1515,...,202041,415085,109402883,0,0,326,1519,846256,38133,935481
2,20201004,56,7383840,96364124,11471,29944,413566,5974,20729,1485,...,201715,413566,108467402,0,0,363,649,824848,38439,955736
3,20201003,56,7345401,95539276,11464,30106,412917,5996,20686,1501,...,201352,412917,107511666,0,0,741,1166,846390,51372,997984
4,20201002,56,7294029,94692886,10813,30697,411751,6106,20612,1533,...,200611,411751,106513682,0,0,835,1342,977186,49534,1127307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,20200126,2,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
255,20200125,2,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
256,20200124,2,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
257,20200123,2,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,1


In [40]:
# Confirm covidtracking_current data has been added
pd.read_sql_query('select * from covidtracking_current', con=engine)

,date,state,positive,probable_cases,negative,pending,total_test_results,hospitalized_currently,hospitalized_cumulative,icu_currently,...,total_tests_antigen,positive_tests_antigen,positive_increase,negative_increase,total,total_test_results_source,total_test_results_increase,pos_neg,death_increase,hospitalized_increase
0,20201006,AK,9587,0,469787,0,479374,45,0,0,...,0,0,0,0,479374,posNeg,0,479374,0,0
1,20201006,AL,160477,18923,1031293,0,1172847,764,17699,0,...,0,0,764,4702,1191770,posNeg,5334,1191770,21,0
2,20201006,AR,88071,3841,1003441,0,1087671,523,5658,237,...,21856,3300,641,7995,1091512,posNeg,8527,1091512,0,72
3,20201006,AS,0,0,1616,0,1616,0,0,0,...,0,0,0,0,1616,posNeg,0,1616,0,0
4,20201006,AZ,221934,4780,1294273,0,1511427,665,19987,138,...,0,0,864,7733,1516207,posNeg,8554,1516207,6,-2316
5,20201006,CA,828461,0,14601960,0,15430421,3175,0,809,...,0,0,1677,127063,15430421,posNeg,128740,15430421,28,0
6,20201006,CO,73537,5088,889084,0,1444262,325,7693,0,...,0,0,461,6341,962621,totalTestEncountersViral,13722,962621,0,20
7,20201006,CT,59241,2412,1685193,0,1744434,129,11699,0,...,0,0,121,17592,1744434,posNeg,17713,1744434,4,0
8,20201006,DC,15652,0,394377,0,410029,105,0,30,...,0,0,105,8356,410029,posNeg,8461,410029,0,0
9,20201006,DE,21466,1074,276047,0,297513,87,0,20,...,0,0,103,1575,297513,posNeg,1678,297513,1,0
